### runs experiments with PYCARET AutoML framework, using tabular data with features derived from the GCMs forecasts

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

### os
import os
import sys

### datetimes
from datetime import datetime, timedelta

### scipy
import numpy as np
import pandas as pd

### plotting
from matplotlib import pyplot as plt
import matplotlib


### import pycaret classification

In [3]:
import pycaret

In [4]:
pycaret.__version__

'2.3.1'

In [5]:
from pycaret.classification import *

### set the random seed 

In [6]:
np.random.seed(42)

### paths 

In [7]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()


In [8]:
sys.path.append(str(CWD.parents[4].joinpath('code')))

In [9]:
CWD.parents[4].joinpath('code')

PosixPath('/home/nicolasf/research/Smart_Ideas_Final/code')

### parameters for papermill

In [10]:
predictors = 'observed_climate_indices'

max_features_lag = 20

lag_to_realtime = 0

varname_target = 'RAIN'

period_features = 'seasonal'

period_target = 'seasonal'

region = 'NNI'

target_col = 'cat_3'

dpath = '/home/nicolasf/research/Smart_Ideas_Final/outputs/tabular'

In [11]:
dpath = pathlib.Path(dpath)

In [12]:
df = pd.read_csv(dpath.joinpath(f"observed_climate_indices_{period_features}_{max_features_lag}lags_{lag_to_realtime}lag_to_realtime_{region}_{varname_target}_{target_col}_{period_features}.csv"), 
                 index_col=0, 
                 parse_dates=True)

In [13]:
df.head()

,EMI_seasonal_0,EMI_seasonal_1,EMI_seasonal_2,EMI_seasonal_3,EMI_seasonal_4,EMI_seasonal_5,EMI_seasonal_6,EMI_seasonal_7,EMI_seasonal_8,EMI_seasonal_9,EMI_seasonal_10,EMI_seasonal_11,EMI_seasonal_12,EMI_seasonal_13,EMI_seasonal_14,EMI_seasonal_15,EMI_seasonal_16,EMI_seasonal_17,EMI_seasonal_18,EMI_seasonal_19,SAM_seasonal_0,SAM_seasonal_1,SAM_seasonal_2,SAM_seasonal_3,SAM_seasonal_4,SAM_seasonal_5,SAM_seasonal_6,SAM_seasonal_7,SAM_seasonal_8,SAM_seasonal_9,SAM_seasonal_10,SAM_seasonal_11,SAM_seasonal_12,SAM_seasonal_13,SAM_seasonal_14,SAM_seasonal_15,SAM_seasonal_16,SAM_seasonal_17,SAM_seasonal_18,SAM_seasonal_19,SPSD_seasonal_0,SPSD_seasonal_1,SPSD_seasonal_2,SPSD_seasonal_3,SPSD_seasonal_4,SPSD_seasonal_5,SPSD_seasonal_6,SPSD_seasonal_7,SPSD_seasonal_8,SPSD_seasonal_9,SPSD_seasonal_10,SPSD_seasonal_11,SPSD_seasonal_12,SPSD_seasonal_13,SPSD_seasonal_14,SPSD_seasonal_15,SPSD_seasonal_16,SPSD_seasonal_17,SPSD_seasonal_18,SPSD_seasonal_19,NINO34_monthly_seasonal_0,NINO34_monthly_seasonal_1,NINO34_monthly_seasonal_2,NINO34_monthly_seasonal_3,NINO34_monthly_seasonal_4,NINO34_monthly_seasonal_5,NINO34_monthly_seasonal_6,NINO34_monthly_seasonal_7,NINO34_monthly_seasonal_8,NINO34_monthly_seasonal_9,NINO34_monthly_seasonal_10,NINO34_monthly_seasonal_11,NINO34_monthly_seasonal_12,NINO34_monthly_seasonal_13,NINO34_monthly_seasonal_14,NINO34_monthly_seasonal_15,NINO34_monthly_seasonal_16,NINO34_monthly_seasonal_17,NINO34_monthly_seasonal_18,NINO34_monthly_seasonal_19,IOD_seasonal_0,IOD_seasonal_1,IOD_seasonal_2,IOD_seasonal_3,IOD_seasonal_4,IOD_seasonal_5,IOD_seasonal_6,IOD_seasonal_7,IOD_seasonal_8,IOD_seasonal_9,IOD_seasonal_10,IOD_seasonal_11,IOD_seasonal_12,IOD_seasonal_13,IOD_seasonal_14,IOD_seasonal_15,IOD_seasonal_16,IOD_seasonal_17,IOD_seasonal_18,IOD_seasonal_19,M1_seasonal_0,M1_seasonal_1,M1_seasonal_2,M1_seasonal_3,M1_seasonal_4,M1_seasonal_5,M1_seasonal_6,M1_seasonal_7,M1_seasonal_8,M1_seasonal_9,M1_seasonal_10,M1_seasonal_11,M1_seasonal_12,M1_seasonal_13,M1_seasonal_14,M1_seasonal_15,M1_seasonal_16,M1_seasonal_17,M1_seasonal_18,M1_seasonal_19,Z1_seasonal_0,Z1_seasonal_1,Z1_seasonal_2,Z1_seasonal_3,Z1_seasonal_4,Z1_seasonal_5,Z1_seasonal_6,Z1_seasonal_7,Z1_seasonal_8,Z1_seasonal_9,Z1_seasonal_10,Z1_seasonal_11,Z1_seasonal_12,Z1_seasonal_13,Z1_seasonal_14,Z1_seasonal_15,Z1_seasonal_16,Z1_seasonal_17,Z1_seasonal_18,Z1_seasonal_19,cat_3
1981-01-31,0.111612,0.150942,0.261013,0.396177,0.407597,0.428216,0.507713,0.525759,0.514545,0.385050,0.209969,0.013183,-0.108046,-0.077418,-0.019172,0.020446,0.061373,0.141795,0.132406,0.067251,-0.707000,-1.547333,-0.885000,-0.712333,-0.753667,-1.323667,-1.490667,-0.950333,-0.792667,-0.607000,-0.404667,0.122000,0.445000,1.195667,1.552667,1.612000,1.034000,0.767000,0.644333,0.488000,-0.598552,-1.002239,-1.195184,-1.339485,-1.587008,-1.632382,-1.285392,-0.661187,-0.207694,-0.332337,-0.481632,-0.411605,-0.118170,0.083293,-0.022189,-0.101691,-0.009978,0.066106,-0.152795,-0.332415,-0.284778,-0.196667,0.015444,0.199444,0.195778,0.104556,0.114667,0.277111,0.435000,0.479667,0.356111,0.260667,0.111889,-0.060000,-0.201222,-0.213889,-0.054222,0.011222,-0.018667,-0.106222,-0.360465,-0.321080,-0.271744,-0.117206,0.269570,0.379849,0.202009,0.022983,0.043747,-0.455868,-0.689808,-0.806166,-0.551322,-0.270038,0.371534,0.576727,0.555993,0.033624,-0.166956,-0.057314,-0.353508,0.576519,0.279468,-0.016427,-0.547122,-0.351573,-0.146520,-0.162645,-0.237880,-0.651597,-1.010304,-0.733880,0.070740,0.165309,-0.713841,-0.648147,-0.727037,-0.256638,-0.309515,0.230830,1.143411,1.272046,0.175163,-0.289267,-0.562220,-0.170917,-0.296536,0.155731,0.416995,0.585679,0.024798,-0.114518,-0.531957,-0.269533,-0.750662,-0.436681,-0.073873,-0.264117,0.298626,0.297327,2
1981-02-28,0.172888,0.111612,0.150942,0.261013,0.396177,0.407597,0.428216,0.507713,0.525759,0.514545,0.385050,0.209969,0.013183,-0.108046,-0.077418,-0.019172,0.020446,0.061373,0.141795,0.132406,-1.045667,-0.707000,-1.547333,-0.885000,-0.712333,-0.753667,-1.323667,-1.490667

In [14]:
df.tail()

,EMI_seasonal_0,EMI_seasonal_1,EMI_seasonal_2,EMI_seasonal_3,EMI_seasonal_4,EMI_seasonal_5,EMI_seasonal_6,EMI_seasonal_7,EMI_seasonal_8,EMI_seasonal_9,EMI_seasonal_10,EMI_seasonal_11,EMI_seasonal_12,EMI_seasonal_13,EMI_seasonal_14,EMI_seasonal_15,EMI_seasonal_16,EMI_seasonal_17,EMI_seasonal_18,EMI_seasonal_19,SAM_seasonal_0,SAM_seasonal_1,SAM_seasonal_2,SAM_seasonal_3,SAM_seasonal_4,SAM_seasonal_5,SAM_seasonal_6,SAM_seasonal_7,SAM_seasonal_8,SAM_seasonal_9,SAM_seasonal_10,SAM_seasonal_11,SAM_seasonal_12,SAM_seasonal_13,SAM_seasonal_14,SAM_seasonal_15,SAM_seasonal_16,SAM_seasonal_17,SAM_seasonal_18,SAM_seasonal_19,SPSD_seasonal_0,SPSD_seasonal_1,SPSD_seasonal_2,SPSD_seasonal_3,SPSD_seasonal_4,SPSD_seasonal_5,SPSD_seasonal_6,SPSD_seasonal_7,SPSD_seasonal_8,SPSD_seasonal_9,SPSD_seasonal_10,SPSD_seasonal_11,SPSD_seasonal_12,SPSD_seasonal_13,SPSD_seasonal_14,SPSD_seasonal_15,SPSD_seasonal_16,SPSD_seasonal_17,SPSD_seasonal_18,SPSD_seasonal_19,NINO34_monthly_seasonal_0,NINO34_monthly_seasonal_1,NINO34_monthly_seasonal_2,NINO34_monthly_seasonal_3,NINO34_monthly_seasonal_4,NINO34_monthly_seasonal_5,NINO34_monthly_seasonal_6,NINO34_monthly_seasonal_7,NINO34_monthly_seasonal_8,NINO34_monthly_seasonal_9,NINO34_monthly_seasonal_10,NINO34_monthly_seasonal_11,NINO34_monthly_seasonal_12,NINO34_monthly_seasonal_13,NINO34_monthly_seasonal_14,NINO34_monthly_seasonal_15,NINO34_monthly_seasonal_16,NINO34_monthly_seasonal_17,NINO34_monthly_seasonal_18,NINO34_monthly_seasonal_19,IOD_seasonal_0,IOD_seasonal_1,IOD_seasonal_2,IOD_seasonal_3,IOD_seasonal_4,IOD_seasonal_5,IOD_seasonal_6,IOD_seasonal_7,IOD_seasonal_8,IOD_seasonal_9,IOD_seasonal_10,IOD_seasonal_11,IOD_seasonal_12,IOD_seasonal_13,IOD_seasonal_14,IOD_seasonal_15,IOD_seasonal_16,IOD_seasonal_17,IOD_seasonal_18,IOD_seasonal_19,M1_seasonal_0,M1_seasonal_1,M1_seasonal_2,M1_seasonal_3,M1_seasonal_4,M1_seasonal_5,M1_seasonal_6,M1_seasonal_7,M1_seasonal_8,M1_seasonal_9,M1_seasonal_10,M1_seasonal_11,M1_seasonal_12,M1_seasonal_13,M1_seasonal_14,M1_seasonal_15,M1_seasonal_16,M1_seasonal_17,M1_seasonal_18,M1_seasonal_19,Z1_seasonal_0,Z1_seasonal_1,Z1_seasonal_2,Z1_seasonal_3,Z1_seasonal_4,Z1_seasonal_5,Z1_seasonal_6,Z1_seasonal_7,Z1_seasonal_8,Z1_seasonal_9,Z1_seasonal_10,Z1_seasonal_11,Z1_seasonal_12,Z1_seasonal_13,Z1_seasonal_14,Z1_seasonal_15,Z1_seasonal_16,Z1_seasonal_17,Z1_seasonal_18,Z1_seasonal_19,cat_3
2019-08-31,0.387744,0.445640,0.431095,0.372740,0.364841,0.450154,0.521218,0.532342,0.437007,0.379107,0.283585,0.235679,0.135573,0.042357,-0.090808,-0.153541,-0.141693,-0.111269,-0.094222,-0.119392,0.472000,0.193667,0.307333,0.369000,0.866000,0.817000,0.993000,0.548333,0.497333,0.007333,0.096000,-0.418333,-0.367333,0.005333,0.819000,1.100000,1.010000,0.395667,0.499667,0.497333,1.634662,1.457499,1.247428,1.230000,1.160006,1.014168,0.895760,0.972053,0.986798,0.861158,0.696149,0.821902,1.242388,1.773674,2.117680,2.169737,1.833288,1.391804,0.974480,0.838313,0.751222,0.768000,0.737111,0.761667,0.849667,0.952778,0.797333,0.505222,0.270000,0.175444,0.112778,-0.100889,-0.412111,-0.642000,-0.836222,-0.898333,-0.937000,-0.783889,-0.616000,-0.364778,0.961243,0.263980,0.148439,0.422641,0.898550,1.151276,1.374914,1.249897,1.122784,0.650992,0.441797,0.219992,0.180442,0.393551,0.404240,0.310248,0.197496,0.463138,0.783337,1.166440,-1.323507,-1.270126,-1.149697,-0.301658,-0.471377,-0.310196,-0.088876,0.104560,-0.506945,-0.612028,-0.294512,0.404091,0.050144,0.206290,-0.315625,-0.307950,-0.879773,-0.754915,-0.254148,-0.041608,0.131026,-0.416729,0.171750,-0.438173,-0.637086,-1.108579,-0.805800,-0.501869,0.604261,0.370162,0.497697,-0.040338,0.370567,0.313655,-0.211313,-0.442698,-0.939586,-0.838544,-0.234679,0.230072,0
2019-09-30,0.401512,0.387744,0.445640,0.431095,0.372740,0.364841,0.450154,0.521218,0.532342,0.437007,0.379107,0.283585,0.235679,0.135573,0.042357,-0.090808,-0.153541,-0.141693,-0.111269,-0.094222,0.712000,0.472000,0.193667,0.307333,0.369000,0.866000,0.817000,0.993000,0.548333,0.497333,0.007333,0.096000,-

In [15]:
df.index

DatetimeIndex(['1981-01-31', '1981-02-28', '1981-03-31', '1981-04-30',
               '1981-05-31', '1981-06-30', '1981-07-31', '1981-08-31',
               '1981-09-30', '1981-10-31',
               ...
               '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', length=468, freq=None)

In [16]:
df.loc[:,'month'] = df.index.month

### set up training and test set 

In [17]:
year_test = 2017

In [18]:
df_test = df.loc[f'{year_test}-04-01':,:]

In [19]:
df_test.head()

,EMI_seasonal_0,EMI_seasonal_1,EMI_seasonal_2,EMI_seasonal_3,EMI_seasonal_4,EMI_seasonal_5,EMI_seasonal_6,EMI_seasonal_7,EMI_seasonal_8,EMI_seasonal_9,EMI_seasonal_10,EMI_seasonal_11,EMI_seasonal_12,EMI_seasonal_13,EMI_seasonal_14,EMI_seasonal_15,EMI_seasonal_16,EMI_seasonal_17,EMI_seasonal_18,EMI_seasonal_19,SAM_seasonal_0,SAM_seasonal_1,SAM_seasonal_2,SAM_seasonal_3,SAM_seasonal_4,SAM_seasonal_5,SAM_seasonal_6,SAM_seasonal_7,SAM_seasonal_8,SAM_seasonal_9,SAM_seasonal_10,SAM_seasonal_11,SAM_seasonal_12,SAM_seasonal_13,SAM_seasonal_14,SAM_seasonal_15,SAM_seasonal_16,SAM_seasonal_17,SAM_seasonal_18,SAM_seasonal_19,SPSD_seasonal_0,SPSD_seasonal_1,SPSD_seasonal_2,SPSD_seasonal_3,SPSD_seasonal_4,SPSD_seasonal_5,SPSD_seasonal_6,SPSD_seasonal_7,SPSD_seasonal_8,SPSD_seasonal_9,SPSD_seasonal_10,SPSD_seasonal_11,SPSD_seasonal_12,SPSD_seasonal_13,SPSD_seasonal_14,SPSD_seasonal_15,SPSD_seasonal_16,SPSD_seasonal_17,SPSD_seasonal_18,SPSD_seasonal_19,NINO34_monthly_seasonal_0,NINO34_monthly_seasonal_1,NINO34_monthly_seasonal_2,NINO34_monthly_seasonal_3,NINO34_monthly_seasonal_4,NINO34_monthly_seasonal_5,NINO34_monthly_seasonal_6,NINO34_monthly_seasonal_7,NINO34_monthly_seasonal_8,NINO34_monthly_seasonal_9,NINO34_monthly_seasonal_10,NINO34_monthly_seasonal_11,NINO34_monthly_seasonal_12,NINO34_monthly_seasonal_13,NINO34_monthly_seasonal_14,NINO34_monthly_seasonal_15,NINO34_monthly_seasonal_16,NINO34_monthly_seasonal_17,NINO34_monthly_seasonal_18,NINO34_monthly_seasonal_19,IOD_seasonal_0,IOD_seasonal_1,IOD_seasonal_2,IOD_seasonal_3,IOD_seasonal_4,IOD_seasonal_5,IOD_seasonal_6,IOD_seasonal_7,IOD_seasonal_8,IOD_seasonal_9,IOD_seasonal_10,IOD_seasonal_11,IOD_seasonal_12,IOD_seasonal_13,IOD_seasonal_14,IOD_seasonal_15,IOD_seasonal_16,IOD_seasonal_17,IOD_seasonal_18,IOD_seasonal_19,M1_seasonal_0,M1_seasonal_1,M1_seasonal_2,M1_seasonal_3,M1_seasonal_4,M1_seasonal_5,M1_seasonal_6,M1_seasonal_7,M1_seasonal_8,M1_seasonal_9,M1_seasonal_10,M1_seasonal_11,M1_seasonal_12,M1_seasonal_13,M1_seasonal_14,M1_seasonal_15,M1_seasonal_16,M1_seasonal_17,M1_seasonal_18,M1_seasonal_19,Z1_seasonal_0,Z1_seasonal_1,Z1_seasonal_2,Z1_seasonal_3,Z1_seasonal_4,Z1_seasonal_5,Z1_seasonal_6,Z1_seasonal_7,Z1_seasonal_8,Z1_seasonal_9,Z1_seasonal_10,Z1_seasonal_11,Z1_seasonal_12,Z1_seasonal_13,Z1_seasonal_14,Z1_seasonal_15,Z1_seasonal_16,Z1_seasonal_17,Z1_seasonal_18,Z1_seasonal_19,cat_3,month
2017-04-30,-0.553659,-0.548826,-0.636523,-0.555749,-0.408841,-0.229576,-0.053188,0.141545,0.284376,0.386566,0.333333,0.345222,0.339860,0.378166,0.310402,0.274053,0.205757,0.183075,0.154188,0.306552,-1.067000,-0.798667,0.216000,0.472333,0.667000,0.744333,0.994667,0.891333,0.715667,1.076000,1.507667,0.808667,0.676000,0.155333,0.355667,0.478000,1.094000,1.150333,0.935000,0.718667,-0.469811,0.319863,0.636402,0.509290,0.391176,0.361184,0.270075,0.024914,-0.288102,-0.588667,-1.261239,-1.634804,-1.769124,-1.371935,-1.087136,-0.836697,-0.670742,-0.542992,-0.357316,0.009623,-0.527000,-0.613889,-0.656000,-0.611444,-0.500000,-0.271222,0.052778,0.509111,1.031222,1.648000,2.163778,2.508333,2.683000,2.632778,2.467333,2.188556,1.910000,1.612111,1.306111,1.052444,-0.427697,-0.898735,-1.375086,-1.485935,-1.805012,-1.671252,-1.623101,-0.925227,-0.547315,-0.516288,-0.532230,-0.302823,0.406996,1.119538,1.649016,1.736251,1.533323,1.157200,0.822365,0.560828,0.792532,-0.105980,-0.182078,-0.644824,-0.314611,-0.825768,-0.711962,-0.983016,-0.630729,-0.722331,-0.168724,0.309747,0.093603,0.457733,0.747092,1.068916,0.322588,0.492964,0.446495,1.038929,1.448154,0.693932,0.038310,-0.651883,0.158221,0.713064,1.676693,0.912949,0.692318,0.272718,0.014909,-0.150415,-0.361118,0.094934,-0.077704,-0.067541,-0.054062,0.921144,1.234918,1.297095,2,4
2017-05-31,-0.551680,-0.553659,-0.548826,-0.636523,-0.555749,-0.408841,-0.229576,-0.053188,0.141545,0.284376,0.386566,0.333333,0.345222,0.339860,0.378166,0.310402,0.274053,0.205757,0.183075,0.154188,-0.569333,-1.067000,-0.798667,0.216000,0.472333,0.667000,0.744333,0.994667,0.891333,0.71

In [20]:
df_test.tail()

,EMI_seasonal_0,EMI_seasonal_1,EMI_seasonal_2,EMI_seasonal_3,EMI_seasonal_4,EMI_seasonal_5,EMI_seasonal_6,EMI_seasonal_7,EMI_seasonal_8,EMI_seasonal_9,EMI_seasonal_10,EMI_seasonal_11,EMI_seasonal_12,EMI_seasonal_13,EMI_seasonal_14,EMI_seasonal_15,EMI_seasonal_16,EMI_seasonal_17,EMI_seasonal_18,EMI_seasonal_19,SAM_seasonal_0,SAM_seasonal_1,SAM_seasonal_2,SAM_seasonal_3,SAM_seasonal_4,SAM_seasonal_5,SAM_seasonal_6,SAM_seasonal_7,SAM_seasonal_8,SAM_seasonal_9,SAM_seasonal_10,SAM_seasonal_11,SAM_seasonal_12,SAM_seasonal_13,SAM_seasonal_14,SAM_seasonal_15,SAM_seasonal_16,SAM_seasonal_17,SAM_seasonal_18,SAM_seasonal_19,SPSD_seasonal_0,SPSD_seasonal_1,SPSD_seasonal_2,SPSD_seasonal_3,SPSD_seasonal_4,SPSD_seasonal_5,SPSD_seasonal_6,SPSD_seasonal_7,SPSD_seasonal_8,SPSD_seasonal_9,SPSD_seasonal_10,SPSD_seasonal_11,SPSD_seasonal_12,SPSD_seasonal_13,SPSD_seasonal_14,SPSD_seasonal_15,SPSD_seasonal_16,SPSD_seasonal_17,SPSD_seasonal_18,SPSD_seasonal_19,NINO34_monthly_seasonal_0,NINO34_monthly_seasonal_1,NINO34_monthly_seasonal_2,NINO34_monthly_seasonal_3,NINO34_monthly_seasonal_4,NINO34_monthly_seasonal_5,NINO34_monthly_seasonal_6,NINO34_monthly_seasonal_7,NINO34_monthly_seasonal_8,NINO34_monthly_seasonal_9,NINO34_monthly_seasonal_10,NINO34_monthly_seasonal_11,NINO34_monthly_seasonal_12,NINO34_monthly_seasonal_13,NINO34_monthly_seasonal_14,NINO34_monthly_seasonal_15,NINO34_monthly_seasonal_16,NINO34_monthly_seasonal_17,NINO34_monthly_seasonal_18,NINO34_monthly_seasonal_19,IOD_seasonal_0,IOD_seasonal_1,IOD_seasonal_2,IOD_seasonal_3,IOD_seasonal_4,IOD_seasonal_5,IOD_seasonal_6,IOD_seasonal_7,IOD_seasonal_8,IOD_seasonal_9,IOD_seasonal_10,IOD_seasonal_11,IOD_seasonal_12,IOD_seasonal_13,IOD_seasonal_14,IOD_seasonal_15,IOD_seasonal_16,IOD_seasonal_17,IOD_seasonal_18,IOD_seasonal_19,M1_seasonal_0,M1_seasonal_1,M1_seasonal_2,M1_seasonal_3,M1_seasonal_4,M1_seasonal_5,M1_seasonal_6,M1_seasonal_7,M1_seasonal_8,M1_seasonal_9,M1_seasonal_10,M1_seasonal_11,M1_seasonal_12,M1_seasonal_13,M1_seasonal_14,M1_seasonal_15,M1_seasonal_16,M1_seasonal_17,M1_seasonal_18,M1_seasonal_19,Z1_seasonal_0,Z1_seasonal_1,Z1_seasonal_2,Z1_seasonal_3,Z1_seasonal_4,Z1_seasonal_5,Z1_seasonal_6,Z1_seasonal_7,Z1_seasonal_8,Z1_seasonal_9,Z1_seasonal_10,Z1_seasonal_11,Z1_seasonal_12,Z1_seasonal_13,Z1_seasonal_14,Z1_seasonal_15,Z1_seasonal_16,Z1_seasonal_17,Z1_seasonal_18,Z1_seasonal_19,cat_3,month
2019-08-31,0.387744,0.445640,0.431095,0.372740,0.364841,0.450154,0.521218,0.532342,0.437007,0.379107,0.283585,0.235679,0.135573,0.042357,-0.090808,-0.153541,-0.141693,-0.111269,-0.094222,-0.119392,0.472000,0.193667,0.307333,0.369000,0.866000,0.817000,0.993000,0.548333,0.497333,0.007333,0.096000,-0.418333,-0.367333,0.005333,0.819000,1.100000,1.010000,0.395667,0.499667,0.497333,1.634662,1.457499,1.247428,1.230000,1.160006,1.014168,0.895760,0.972053,0.986798,0.861158,0.696149,0.821902,1.242388,1.773674,2.117680,2.169737,1.833288,1.391804,0.974480,0.838313,0.751222,0.768000,0.737111,0.761667,0.849667,0.952778,0.797333,0.505222,0.270000,0.175444,0.112778,-0.100889,-0.412111,-0.642000,-0.836222,-0.898333,-0.937000,-0.783889,-0.616000,-0.364778,0.961243,0.263980,0.148439,0.422641,0.898550,1.151276,1.374914,1.249897,1.122784,0.650992,0.441797,0.219992,0.180442,0.393551,0.404240,0.310248,0.197496,0.463138,0.783337,1.166440,-1.323507,-1.270126,-1.149697,-0.301658,-0.471377,-0.310196,-0.088876,0.104560,-0.506945,-0.612028,-0.294512,0.404091,0.050144,0.206290,-0.315625,-0.307950,-0.879773,-0.754915,-0.254148,-0.041608,0.131026,-0.416729,0.171750,-0.438173,-0.637086,-1.108579,-0.805800,-0.501869,0.604261,0.370162,0.497697,-0.040338,0.370567,0.313655,-0.211313,-0.442698,-0.939586,-0.838544,-0.234679,0.230072,0,8
2019-09-30,0.401512,0.387744,0.445640,0.431095,0.372740,0.364841,0.450154,0.521218,0.532342,0.437007,0.379107,0.283585,0.235679,0.135573,0.042357,-0.090808,-0.153541,-0.141693,-0.111269,-0.094222,0.712000,0.472000,0.193667,0.307333,0.369000,0.866000,0.817000,0.993000,0.548333,0.497333,0.007333,0.

In [21]:
df = df.loc[:f'{year_test - 1}-12',:]

In [22]:
df.tail()

,EMI_seasonal_0,EMI_seasonal_1,EMI_seasonal_2,EMI_seasonal_3,EMI_seasonal_4,EMI_seasonal_5,EMI_seasonal_6,EMI_seasonal_7,EMI_seasonal_8,EMI_seasonal_9,EMI_seasonal_10,EMI_seasonal_11,EMI_seasonal_12,EMI_seasonal_13,EMI_seasonal_14,EMI_seasonal_15,EMI_seasonal_16,EMI_seasonal_17,EMI_seasonal_18,EMI_seasonal_19,SAM_seasonal_0,SAM_seasonal_1,SAM_seasonal_2,SAM_seasonal_3,SAM_seasonal_4,SAM_seasonal_5,SAM_seasonal_6,SAM_seasonal_7,SAM_seasonal_8,SAM_seasonal_9,SAM_seasonal_10,SAM_seasonal_11,SAM_seasonal_12,SAM_seasonal_13,SAM_seasonal_14,SAM_seasonal_15,SAM_seasonal_16,SAM_seasonal_17,SAM_seasonal_18,SAM_seasonal_19,SPSD_seasonal_0,SPSD_seasonal_1,SPSD_seasonal_2,SPSD_seasonal_3,SPSD_seasonal_4,SPSD_seasonal_5,SPSD_seasonal_6,SPSD_seasonal_7,SPSD_seasonal_8,SPSD_seasonal_9,SPSD_seasonal_10,SPSD_seasonal_11,SPSD_seasonal_12,SPSD_seasonal_13,SPSD_seasonal_14,SPSD_seasonal_15,SPSD_seasonal_16,SPSD_seasonal_17,SPSD_seasonal_18,SPSD_seasonal_19,NINO34_monthly_seasonal_0,NINO34_monthly_seasonal_1,NINO34_monthly_seasonal_2,NINO34_monthly_seasonal_3,NINO34_monthly_seasonal_4,NINO34_monthly_seasonal_5,NINO34_monthly_seasonal_6,NINO34_monthly_seasonal_7,NINO34_monthly_seasonal_8,NINO34_monthly_seasonal_9,NINO34_monthly_seasonal_10,NINO34_monthly_seasonal_11,NINO34_monthly_seasonal_12,NINO34_monthly_seasonal_13,NINO34_monthly_seasonal_14,NINO34_monthly_seasonal_15,NINO34_monthly_seasonal_16,NINO34_monthly_seasonal_17,NINO34_monthly_seasonal_18,NINO34_monthly_seasonal_19,IOD_seasonal_0,IOD_seasonal_1,IOD_seasonal_2,IOD_seasonal_3,IOD_seasonal_4,IOD_seasonal_5,IOD_seasonal_6,IOD_seasonal_7,IOD_seasonal_8,IOD_seasonal_9,IOD_seasonal_10,IOD_seasonal_11,IOD_seasonal_12,IOD_seasonal_13,IOD_seasonal_14,IOD_seasonal_15,IOD_seasonal_16,IOD_seasonal_17,IOD_seasonal_18,IOD_seasonal_19,M1_seasonal_0,M1_seasonal_1,M1_seasonal_2,M1_seasonal_3,M1_seasonal_4,M1_seasonal_5,M1_seasonal_6,M1_seasonal_7,M1_seasonal_8,M1_seasonal_9,M1_seasonal_10,M1_seasonal_11,M1_seasonal_12,M1_seasonal_13,M1_seasonal_14,M1_seasonal_15,M1_seasonal_16,M1_seasonal_17,M1_seasonal_18,M1_seasonal_19,Z1_seasonal_0,Z1_seasonal_1,Z1_seasonal_2,Z1_seasonal_3,Z1_seasonal_4,Z1_seasonal_5,Z1_seasonal_6,Z1_seasonal_7,Z1_seasonal_8,Z1_seasonal_9,Z1_seasonal_10,Z1_seasonal_11,Z1_seasonal_12,Z1_seasonal_13,Z1_seasonal_14,Z1_seasonal_15,Z1_seasonal_16,Z1_seasonal_17,Z1_seasonal_18,Z1_seasonal_19,cat_3,month
2016-08-31,0.284376,0.386566,0.333333,0.345222,0.339860,0.378166,0.310402,0.274053,0.205757,0.183075,0.154188,0.306552,0.541483,0.741089,0.734737,0.590485,0.429107,0.333613,0.250029,0.107014,0.715667,1.076000,1.507667,0.808667,0.676000,0.155333,0.355667,0.478000,1.094000,1.150333,0.935000,0.718667,0.739333,1.006333,0.888333,1.071333,0.492667,0.254667,-0.559000,-0.405667,-0.288102,-0.588667,-1.261239,-1.634804,-1.769124,-1.371935,-1.087136,-0.836697,-0.670742,-0.542992,-0.357316,0.009623,0.359882,0.366412,0.225432,-0.151937,-0.245651,-0.238613,0.027170,0.272018,1.031222,1.648000,2.163778,2.508333,2.683000,2.632778,2.467333,2.188556,1.910000,1.612111,1.306111,1.052444,0.787889,0.584667,0.480444,0.558333,0.693000,0.689444,0.534000,0.251889,-0.547315,-0.516288,-0.532230,-0.302823,0.406996,1.119538,1.649016,1.736251,1.533323,1.157200,0.822365,0.560828,0.220351,-0.188146,-0.586598,-0.609012,-0.211202,0.225568,0.048293,-0.428970,-0.630729,-0.722331,-0.168724,0.309747,0.093603,0.457733,0.747092,1.068916,0.322588,0.492964,0.446495,1.038929,0.303118,0.527342,-0.453899,-0.851282,-0.487269,0.315220,1.073972,0.957333,0.692318,0.272718,0.014909,-0.150415,-0.361118,0.094934,-0.077704,-0.067541,-0.054062,0.921144,1.234918,1.297095,0.639690,0.298621,-0.315683,-0.591929,-0.094844,0.420969,0.554348,-0.151342,1,8
2016-09-30,0.141545,0.284376,0.386566,0.333333,0.345222,0.339860,0.378166,0.310402,0.274053,0.205757,0.183075,0.154188,0.306552,0.541483,0.741089,0.734737,0.590485,0.429107,0.333613,0.250029,0.891333,0.715667,1.076000,1.507667,0.808667,0.676000,0.155333,0.355667,0.478000,1.094000,1.150333,0.935000,0.7

## Set up the pycaret experiment 

In [23]:
exp_mclf = setup(data = df, 
                 target = target_col, 
                 categorical_features=['month'], 
                 normalize=True, 
                 normalize_method='zscore', 
                 remove_multicollinearity=True, 
                 multicollinearity_threshold = 0.3, 
                 feature_selection = True, 
                 feature_selection_threshold=0.1, 
                 feature_selection_method = 'boruta', 
                 fold_strategy ='kfold', 
                 fold =3, 
                 session_id=42, silent=True)

,Description,Value
0,session_id,123
1,Target,cat_3
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2"
4,Original Data,"(432, 142)"
5,Missing Values,False
6,Numeric Features,140
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


### compare the models 

In [24]:
best = compare_models(n_select=10, turbo=True, sort='Accuracy')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.4770,0.6376,0.4781,0.4900,0.4772,0.2164,0.2190,0.0067
et,Extra Trees Classifier,0.4736,0.6548,0.4766,0.4928,0.4754,0.2145,0.2177,0.2800
rf,Random Forest Classifier,0.4636,0.6583,0.4651,0.4813,0.4598,0.2006,0.2060,0.3133
nb,Naive Bayes,0.4603,0.6394,0.4636,0.4734,0.4613,0.1934,0.1958,0.3433
lightgbm,Light Gradient Boosting Machine,0.4570,0.6315,0.4620,0.4668,0.4516,0.1889,0.1931,0.1800
knn,K Neighbors Classifier,0.4537,0.6205,0.4517,0.4539,0.4416,0.1791,0.1835,0.4233
catboost,CatBoost Classifier,0.4537,0.6545,0.4573,0.4675,0.4529,0.1859,0.1891,1.2733
lda,Linear Discriminant Analysis,0.4502,0.6241,0.4533,0.4644,0.4470,0.1787,0.1832,0.0100
gbc,Gradient Boosting Classifier,0.4469,0.6148,0.4454,0.4507,0.4421,0.1693,0.1720,0.0833
ridge,Ridge Classifier,0.4437,0.0000,0.4487,0.4587,0.4366,0.1700,0.1760,0.0067


### pull the above dataframe 

In [25]:
models_comparison = pull() 

In [26]:
models_comparison.to_csv(f'./logging/pycaret_experiment_compare_models_{predictors}_{period_features}_features_{max_features_lag}lags_{lag_to_realtime}lag_to_realtime_{varname_target}_target_{target_col}_{region}_{period_target}.csv')

In [27]:
acc_models = {}

In [28]:
for b in best: 
    
    model = create_model(b)
    
    final_model = finalize_model(model)
    
    test_predictions = predict_model(final_model, data=df_test)
    
    acc = (test_predictions[target_col] == test_predictions.Label).sum() / len(test_predictions)
    
    model_name = str(model).split('(')[0]
    
    if 'catboost' in model_name: 
        
        model_name = 'CatBoostClassifier'

    acc_models[model_name] = acc
    
#     print(f"ACC on test set (2017-2019): {str(model).split('(')[0]}: {acc:+4.2f}")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.4158,0.0000,0.4155,0.4305,0.4132,0.1252,0.1286
1,0.4851,0.0000,0.4923,0.5088,0.4828,0.2338,0.2416
2,0.4300,0.0000,0.4385,0.4367,0.4137,0.1510,0.1579
Mean,0.4437,0.0000,0.4487,0.4587,0.4366,0.1700,0.1760
SD,0.0299,0.0000,0.0322,0.0355,0.0327,0.0463,0.0479


In [29]:
df_acc_models = pd.Series(acc_models).to_frame(name='ACC')

In [30]:
df_acc_models

,ACC
QuadraticDiscriminantAnalysis,0.333333
ExtraTreesClassifier,0.333333
RandomForestClassifier,0.333333
GaussianNB,0.424242
LGBMClassifier,0.363636
KNeighborsClassifier,0.393939
CatBoostClassifier,0.424242
LinearDiscriminantAnalysis,0.393939
GradientBoostingClassifier,0.484848
RidgeClassifier,0.393939


In [31]:
N = 3

In [32]:
blend = blend_models(best[:N])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.4257,0.6218,0.4290,0.4374,0.4242,0.1438,0.1463
1,0.5248,0.7049,0.5285,0.5313,0.5246,0.2894,0.2915
2,0.4500,0.6696,0.4525,0.4748,0.4545,0.1806,0.1836
Mean,0.4668,0.6654,0.4700,0.4812,0.4678,0.2046,0.2071
SD,0.0421,0.0341,0.0425,0.0386,0.0420,0.0618,0.0615


In [33]:
blend_final = finalize_model(blend)

In [34]:
test_predictions_blend = predict_model(blend_final, data=df_test)

In [35]:
acc = (test_predictions_blend[target_col] == test_predictions_blend.Label).sum() / len(df_test)

In [36]:
df_acc_models.append(pd.Series({'blend':acc}).to_frame(name='ACC'))

,ACC
QuadraticDiscriminantAnalysis,0.333333
ExtraTreesClassifier,0.333333
RandomForestClassifier,0.333333
GaussianNB,0.424242
LGBMClassifier,0.363636
KNeighborsClassifier,0.393939
CatBoostClassifier,0.424242
LinearDiscriminantAnalysis,0.393939
GradientBoostingClassifier,0.484848
RidgeClassifier,0.393939


In [42]:
df_acc_models = df_acc_models.sort_values(by='ACC', ascending=False)

In [43]:
df_acc_models

,ACC
GradientBoostingClassifier,0.484848
GaussianNB,0.424242
CatBoostClassifier,0.424242
KNeighborsClassifier,0.393939
LinearDiscriminantAnalysis,0.393939
RidgeClassifier,0.393939
LGBMClassifier,0.363636
QuadraticDiscriminantAnalysis,0.333333
ExtraTreesClassifier,0.333333
RandomForestClassifier,0.333333


### saves to disk the ACC results on the test set 

In [49]:
pd.value_counts(df_test.loc[:,target_col])

0    12
1    11
2    10
Name: cat_3, dtype: int64

In [37]:
df_acc_models.to_csv(f'./logging/pycaret_experiment_test_set_ACC_blend_best_{predictors}_{period_features}_features_{max_features_lag}lags_{lag_to_realtime}lag_to_realtime_{varname_target}_target_{target_col}_{region}_{period_target}.csv')